## Variables
#### Buses
* bus: ($ch_n, st_n, r_{n,m}, tb_{n,m}, sb_{n,m}$)
    * $ch_n$ - charging state of bus n
    * $st_n$ - state of bus n (3 states: deployed, not-deployed, charging/refueling)
    * $r_{n,m}$ - the route on which the bus is deployed
    * $tb_{n,m}$ - the previous stop the bus travelled to (travel time)
    * $sb_{n,m}$ - the previous stop which was served by the bus (service time)
        * If $tb_n$= $sb_n$, generate travel time for $tb_{n+1}$; $tb_n$ = $tb_{n+1}$ if accepted
        * Else if $tb_n$> $sb_n$, generate service time for $sb_{n+1}$; $sb_n$ = $sb_{n+1}$ if accepted
    * Arrival time - route generated arrays
    * Service Time - route generated arrays

___
### System State New
SS = (n_buses, curr_dem, buses_deployed, buses_recharge, buses_standstill)
SS = (n_buses, curr_dem, buses_deployed, buses_recharge, buses_standstill, route_1_buses, route_2_buses,....)

#### Monitoring System State 

| Time | System State | Bus | Charge | Route | State | Event | Process Time | Demand-Current | Demand-Actual | Demand-Charge |
| ---- | ------------ | --- | ------ | ----- | ----- | ----- | ------------ | -------------- | ------------- | ------------- |
| 0    | (1,1,1,0,0)  | 1   | 50     | '1'   | 1     | 0     | 0            | 0              | 0             | 15            |
| 10.3 | (1,0,1,0,0)  | 1   | 48     | '1'   | 1     | 1     | 10.3         | -              | -             | -             |
| 12.3 | (1,0,1,0,0)  | 1   | 47.5   | '1'   | 1     | 0     | 2.0          | -              | -             | -             |

___

#### Monitoring Bus Deployments

| Time | Demand-Current | Demand-Actual | Demand-Charge | Bus | Charge | Route | Event Array  | Time Array   |
| ---- | -------------- | ------------- | ------------- | --- | ------ | ----- | ------------ | ------------ |
| 0    | 0              | 0             | 15            | 1   | 50     | '1'   | [1, 0, 1]    | [12, 35, 41] |
| 40   | 41             | 40            | 20            | 1   | 35     | '2'   | [1, 0, 1]    | [48, 50, 55] |



In [1]:
import pandas as pd
import numpy as np
import numpy.random as random
from route_functions import *
import matplotlib.pyplot as plt
import scipy.stats as st
import statsmodels.api as sm 

# to ignore warning on calculations
import warnings
warnings.filterwarnings("ignore")

In [2]:
path = r"C:\Users\krishrao\Desktop\Laptop Backup\Krishna\Fall'21\IOE 574\Term Project\IOE574_Project\Saved files\\"
varred = pd.read_parquet(path+'bd_table_refill_10_40_0.075_0.065.parquet')

In [3]:
# intializing route table
routes = pd.read_excel('Model_Parameters.xlsx', 'Routes')

n_buses = 10                            # number of buses

# For liquid fuel buses
running_consumption = 0.075             # fuel(lts.)/kWh per minute
service_consumption = 0.065             # fuel(lts.)/kWh per minute
refuel_consumption = -30                # -30 for fuel, -2 for electric
tank_size = 150                         # 150 lts. / 240 kWh 
level_mean = 40                         # normal distribution.. can be changed
level_std = 0                           # currently constant at level_mean
refuel = 'refill'                       # 'refill', recharge'


# For electric buses
running_consumption = 0.5              # fuel(lts.)/kWh per minute
service_consumption = 0.3              # fuel(lts.)/kWh per minute
refuel_consumption = -2                # -30 for fuel, -2 for electric
tank_size = 240                         # 150 lts. / 240 kWh 
level_mean = 80                         # normal distribution.. can be changed
level_std = 0                           # currently constant at level_mean
refuel = 'recharge'                     # 'refill', recharge'


#-----
# customizable parameters
#average_bus_speed = 30 miles /hr
refuel_stations = 2
conversion_factor = 100/tank_size

# cost rates
emp_rate = 15                           # per hour basis
fuel_rate = 0.882                       # dollar 0.882 per lt. / 0.1275 per kWhr
delay_rate = 2.4                        # dollar per min ** reason
maintain_rate = 1.67                    # dollar per min 1.67 for fuel, 2.25 for elctric


# setting up initial values for simulation
n_routes = 4                            # number of routes
SimTime = 720

SS_cols = ['Time', 'System_State', 'Bus', 'Charge', 'Route', 
           'State', 'Event', 'Process_Time', 'Demand_Current', 
           'Demand_Actual', 'Demand_Charge']
BD_cols = ['Time', 'Demand_Current', 'Demand_Actual', 'Demand_Charge', 
           'Bus', 'Charge', 'Route', 'Event_Array', 'Time_Array']

In [4]:
replicates = int(varred['Replication'].max())
ss_table = pd.DataFrame(columns=SS_cols+['Replication'])
bd_table = pd.DataFrame(columns=BD_cols+['Replication'])

for i in range(replicates):
    print('-----\nRunning buses at Day ', i+1)                     # intializing required variabes
    event_array = list(varred['Event_Array'][(varred['Replication']==(i+1))&(varred['Route']!='refill')])
    time_array = list(varred['Time_Array'][(varred['Replication']==(i+1))&(varred['Route']!='refill')])

    # intializing required variabes
    buses = [bus(level_mean, level_std) for i in range(n_buses)]    # fleet of buses
    t = 0                                      # start time of simulation
    T = SimTime

    # updating demands
    demand_at, demand_r, demand_c = gen_demands(n_routes, T)
    demand_ct = demand_at.copy()         # demand current time and demand actual time
    dct_flag = 0                         # demand at current time

    ss_tab, bd_tab = fleet_simulation_varred(t, T, routes, event_array, time_array, buses, refuel, 
                                             running_consumption, service_consumption, refuel_stations, 
                                             refuel_consumption, conversion_factor, demand_at, demand_ct, 
                                             demand_r, demand_c, dct_flag, SS_cols, BD_cols)
    ss_tab['Replication'] = i+1
    bd_tab['Replication'] = i+1
    ss_table = ss_table.append(ss_tab, ignore_index=True)
    bd_table = bd_table.append(bd_tab, ignore_index=True)

-----
Running buses at Day  1
-----
Running buses at Day  2
-----
Running buses at Day  3
-----
Running buses at Day  4
-----
Running buses at Day  5
-----
Running buses at Day  6
-----
Running buses at Day  7
-----
Running buses at Day  8
-----
Running buses at Day  9
-----
Running buses at Day  10
-----
Running buses at Day  11
-----
Running buses at Day  12
-----
Running buses at Day  13
-----
Running buses at Day  14
-----
Running buses at Day  15


In [5]:
ss_table

,Time,System_State,Bus,Charge,Route,State,Event,Process_Time,Demand_Current,Demand_Actual,Demand_Charge,Replication
0,NaN,"[10, 0, 0, 0, 10]",NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,1
1,0.00,"[10, 3, 1, 0, 9]",10,80.000000,1,1,0,0.00,0.0,0.0,25.0,1
2,0.00,"[10, 2, 2, 0, 8]",9,80.000000,2,1,0,0.00,0.0,0.0,25.0,1
3,0.00,"[10, 1, 3, 0, 7]",8,80.000000,3,1,0,0.00,0.0,0.0,25.0,1
4,0.32,"[10, 0, 3, 0, 7]",9,79.933000,2,1,1.0,0.32,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...
64713,754.60,"[10, 0, 1, 0, 9]",9,61.530750,2,1,0.0,0.74,NaN,NaN,NaN,15
64714,756.24,"[10, 0, 1, 0, 9]",9,61.189083,2,1,1.0,1.64,NaN,NaN,NaN,15
64715,756.92,"[10, 0, 1, 0, 9]",9,61.104083,2,1,0.0,0.68,NaN,NaN,NaN,15
64716,757.98,"[10, 0, 1, 0, 9]",9,60.883250,2,1,1.0,1.06,NaN,NaN,NaN,15


In [6]:
bd_table

,Time,Demand_Current,Demand_Actual,Demand_Charge,Bus,Charge,Route,Event_Array,Time_Array,Replication,State
0,0.00,0.00,0.0,25.0,10,80.000000,1,"[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, ...","[1.06, 2.43, 2.48, 4.01, 6.13, 6.5, 11.0, 11.2...",1,1.0
1,0.00,0.00,0.0,25.0,9,80.000000,2,"[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, ...","[0.32, 0.59, 5.79, 6.41, 8.55, 9.58, 11.38, 12...",1,1.0
2,0.00,0.00,0.0,25.0,8,80.000000,3,"[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, ...","[4.82, 5.7, 8.41, 9.24, 13.59, 13.82, 15.24, 1...",1,1.0
3,15.00,15.00,15.0,25.0,7,80.000000,1,"[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, ...","[16.2, 17.07, 17.27, 17.64, 20.95, 21.44, 25.8...",1,1.0
4,30.00,30.00,30.0,25.0,6,80.000000,1,"[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, ...","[33.18, 35.01, 36.64, 36.95, 39.88, 40.53, 44....",1,1.0
...,...,...,...,...,...,...,...,...,...,...,...
2052,702.88,702.88,680.0,25.0,7,60.805000,4,"[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, ...","[681.57, 682.18, 684.06, 684.52, 686.24, 686.8...",15,1.0
2053,700.35,700.35,690.0,25.0,7,61.734000,1,"[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, ...","[691.29, 691.36, 692.09, 693.65, 695.51, 696.0...",15,1.0
2054,703.12,703.12,700.0,25.0,9,70.876000,2,"[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, ...","[701.9, 702.7, 706.69, 707.05, 711.0, 711.89, ...",15,1.0
2055,706.34,706.34,700.0,25.0,8,56.354167,4,"[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, ...","[701.57, 701.83, 702.03, 703.79, 706.75, 707.2...",15,1.0


In [7]:
varred

,Time,Demand_Current,Demand_Actual,Demand_Charge,Bus,Charge,Route,Event_Array,Time_Array,Replication,State
0,0.0,0.0,0.0,25.0,10,40.000,1,"[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, ...","[1.06, 2.43, 2.48, 4.01, 6.13, 6.5, 11.0, 11.2...",1,1.0
1,0.0,0.0,0.0,25.0,9,40.000,2,"[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, ...","[0.32, 0.59, 5.79, 6.41, 8.55, 9.58, 11.38, 12...",1,1.0
2,0.0,0.0,0.0,25.0,8,40.000,3,"[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, ...","[4.82, 5.7, 8.41, 9.24, 13.59, 13.82, 15.24, 1...",1,1.0
3,15.0,15.0,15.0,25.0,7,40.000,1,"[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, ...","[16.2, 17.07, 17.27, 17.64, 20.95, 21.44, 25.8...",1,1.0
4,30.0,30.0,30.0,25.0,6,40.000,1,"[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, ...","[33.18, 35.01, 36.64, 36.95, 39.88, 40.53, 44....",1,1.0
...,...,...,...,...,...,...,...,...,...,...,...
2080,680.0,680.0,680.0,25.0,2,61.361,4,"[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, ...","[681.57, 682.18, 684.06, 684.52, 686.24, 686.8...",15,1.0
2081,690.0,690.0,690.0,25.0,10,72.515,1,"[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, ...","[691.29, 691.36, 692.09, 693.65, 695.51, 696.0...",15,1.0
2082,700.0,700.0,700.0,25.0,7,71.306,2,"[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, ...","[701.9, 702.7, 706.69, 707.05, 711.0, 711.89, ...",15,1.0
2083,700.0,700.0,700.0,25.0,5,65.247,4,"[1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, ...","[701.57, 701.83, 702.03, 703.79, 706.75, 707.2...",15,1.0


___
## Model Analysis

In [8]:
# delay analysis on deployments
num_delay = sum(np.array(bd_table['Demand_Current'] - bd_table['Demand_Actual'])>0)
total_delay = sum(np.nan_to_num(np.array(bd_table['Demand_Current'] - bd_table['Demand_Actual'])))
avg_delay = round(np.mean(np.nan_to_num(np.array(bd_table['Demand_Current'] - bd_table['Demand_Actual']))), 3)
std_delay = round(np.std(np.nan_to_num(np.array(bd_table['Demand_Current'] - bd_table['Demand_Actual']))), 3)
max_delay = max(np.nan_to_num(np.array(bd_table['Demand_Current'] - bd_table['Demand_Actual'])))

# cost analysis
run_time, ser_time, ref_time, emp_cost, fuel_cost, delay_cost = cost_analysis(n_buses, replicates, refuel, ss_table, 
                                                                              bd_table, emp_rate, fuel_rate, delay_rate, 
                                                                              refuel_consumption)

In [12]:
print('Delay Analyis')
print('\tTotal number of replications (days) -', replicates)
print('\tTotal number of delays -', num_delay)
print('\tTotal delay in minutes -', round(total_delay, 2))
print('\tAverage of delay events in minutes -', round(total_delay/num_delay, 2))

print('\n\tAverage delay (all events) in minutes -', round(avg_delay, 2))
print('\tStd. Deviation of delay (all events) in minutes -', round(std_delay, 2))
print('\tMaximum delay (all events) in minutes -', round(max_delay, 2))


print('\n\tTotal number of deployments -', bd_table.shape[0])
print('\tTotal number of route deployments -', sum(bd_table['Route']!=refuel))
print('\tTotal number of refills deployments -', sum(bd_table['Route']==refuel))

print('\n\nCost Analysis')
print('\tAverage running time for all buses per day in minutes -', run_time)
print('\tAverage service time for all buses per day in minutes -', ser_time)
print('\tAverage refuel time for all buses per day in minutes -', ref_time)

print('\n\tAverage employeee costs paid per day in dollars -', emp_cost)
print('\tAverage fuel costs paid per day in dollars -', fuel_cost)
print('\tAverage delay costs paid per day in dollars -', delay_cost)
print('\tAverage maintenance costs paid per day in dollars -', round(maintain_rate*(run_time+ser_time+ref_time)))

Delay Analyis
	Total number of replications (days) - 15
	Total number of delays - 957
	Total delay in minutes - 25249.9
	Average of delay events in minutes - 26.38

	Average delay (all events) in minutes - 12.28
	Std. Deviation of delay (all events) in minutes - 17.35
	Maximum delay (all events) in minutes - 78.67

	Total number of deployments - 2057
	Total number of route deployments - 1935
	Total number of refills deployments - 122


Cost Analysis
	Average running time for all buses per day in minutes - 2607.05
	Average service time for all buses per day in minutes - 1506.6
	Average refuel time for all buses per day in minutes - 574.56

	Average employeee costs paid per day in dollars - 1172
	Average fuel costs paid per day in dollars - 1013.52384
	Average delay costs paid per day in dollars - 60600
	Average maintenance costs paid per day in dollars - 7829


In [10]:
# Charge distribution
ss_table['Charge'].min(), ss_table['Charge'].max()

(15.279, 102.31)

In [11]:
ss_table.to_parquet(path+'ss_table_'+refuel+'_'+np.str(n_buses)+'_'+
                    np.str(level_mean)+'_'+np.str(running_consumption)+'_'+
                    np.str(service_consumption)+'.parquet')
bd_table.to_parquet(path+'bd_table_'+refuel+'_'+np.str(n_buses)+'_'+
                    np.str(level_mean)+'_'+np.str(running_consumption)+'_'+
                    np.str(service_consumption)+'.parquet')